In [254]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [255]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# Question 1

In [256]:
fifa = pd.read_csv("/content/drive/MyDrive/FIFA 2023/male_players (legacy).csv")

<ipython-input-256-6ce202a2f62b>:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  fifa = pd.read_csv("/content/drive/MyDrive/FIFA 2023/male_players (legacy).csv")


In [257]:
threshold = int(0.3 * 100)
new_fifa = fifa.dropna(thresh = threshold)

In [258]:
numeric_data= new_fifa.select_dtypes(include=['int64','float64'])

In [259]:
#drops goalkeeping column
new_fifa = new_fifa.drop(columns = ['goalkeeping_speed'])

In [260]:
#Separates data into only integers and floats
numeric_data= new_fifa.select_dtypes(include=['int64','float64'])

In [261]:
#Separates data into only strings
non_numeric = new_fifa.select_dtypes(include = ['object'])

In [262]:
#drops unessecary columns
new_fifa = new_fifa.drop(columns = ['club_loaned_from', 'player_url', 'fifa_update_date', 'player_face_url', 'real_face', 'short_name', 'dob'])

In [264]:
from sklearn.impute import SimpleImputer

In [265]:
sc = SimpleImputer(strategy='median')

In [266]:
#Fills all nas in the numeric data with the median values
numeric_data_filled = sc.fit_transform(numeric_data)

In [267]:
#Creates a dataframe of the data with the filled values
numeric_data_df = pd.DataFrame(numeric_data_filled, columns=numeric_data.columns)

In [268]:
sc1 = SimpleImputer(strategy='most_frequent')

In [269]:
#fills all nas with the most frequent values
non_numeric_filled = sc1.fit_transform(non_numeric)

In [270]:
#creates a dataframe of data with filled values
non_numeric_df = pd.DataFrame(non_numeric_filled, columns=non_numeric.columns)

In [271]:
# Adds the dataframes after filling the nas
Xfilled = pd.concat([non_numeric_df,numeric_data_df], axis = 1)

# Question 2

In [272]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [273]:
categorical_list = non_numeric.columns.tolist()

In [274]:
#encodes all the strings in the data
def encode(column_list):
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(column_list)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)

    return integer_encoded

In [275]:
#encodes all strings
for i in categorical_list:
  Xfilled[i] = Xfilled[i].astype(str)
  Xfilled[i] = Xfilled[i].tolist()
  Xfilled[i] = encode(Xfilled[i])

In [277]:
corr_matrix = Xfilled.corr()

In [278]:
correlations = corr_matrix['overall'].sort_values(ascending=False)

In [279]:
#Creates a file with all the correlations
correlations.to_csv('correlations')

**The correlation cutoff for this model is 0.5. Since the variables do not have have a strong negative correlation,**
**I will be using 10 features with a correlation greater than or equal 0.5 when rounded up. The variables being used are**
* movement_reactions
* potential
* passing
* wage_eur
* mentality_composure
* value_eur
* dribbling
* Attacking_short_passing
* mentality_vision
* international_reputation

In [281]:
x = Xfilled[['movement_reactions','potential','passing','wage_eur',
             'value_eur','dribbling','attacking_short_passing','mentality_composure',
             'mentality_vision', 'international_reputation']]

In [282]:
y = Xfilled['overall']

In [283]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold

In [284]:
n_folds = 10
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

In [285]:
#random_state = Means we want to randomize the data
Xtrain,Xtest,Ytrain,Ytest = train_test_split(x,y,test_size=0.2, random_state=42)

# Question 3

## XGB Regressor

In [286]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score

In [287]:
from sklearn.model_selection import GridSearchCV

In [209]:
# xgb regressor to train model
import xgboost as xgb
model1 = xgb.XGBRegressor()
model1_scores = cross_val_score(model1, Xtrain, Ytrain, cv=kf, scoring='neg_mean_squared_error')
model1.fit(Xtrain, Ytrain)
xgb_test_score = model1.score(Xtest, Ytest)
y_pred = model1.predict(Xtest)
print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,Ytest)},
Mean Squared Error = {mean_squared_error(y_pred,Ytest)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,Ytest))},
R2 Score = {r2_score(y_pred, Ytest)}
          """)
print(f"Test: {xgb_test_score}")


Mean Absolute Error = 1.0751832135225259,
Mean Squared Error = 2.1755292620240234,
Root Mean Squared Error = 1.4749675460917855,
R2 Score = 0.954425974140603
          
Test: 0.9561062548991788


In [210]:
# gradient boosting regressor to train model
from sklearn.ensemble import GradientBoostingRegressor
model2 = GradientBoostingRegressor()
model2_scores = cross_val_score(model2, Xtrain, Ytrain, cv=kf, scoring='neg_mean_squared_error')
model2.fit(Xtrain, Ytrain)
grad_test_score = model2.score(Xtest, Ytest)
y_pred2 = model2.predict(Xtest)
print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred2,Ytest)},
Mean Squared Error = {mean_squared_error(y_pred2,Ytest)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred2,Ytest))},
R2 Score = {r2_score(y_pred2, Ytest)}
          """)
print(f"Test: {grad_test_score}")


Mean Absolute Error = 1.326467939273926,
Mean Squared Error = 3.0826177569316786,
Root Mean Squared Error = 1.7557385217997805,
R2 Score = 0.932559141900828
          
Test: 0.9378047262209015


## Grid Search

In [289]:
parameters = {
    'gb__n_estimators': [100, 200],
    'gb__max_depth': [3, 4, 5],
    'gb__learning_rate': [0.01, 0.1],
}
grid_search = GridSearchCV(estimator = xgb.XGBRegressor(), param_grid = parameters, cv=5, scoring='neg_mean_squared_error')

In [291]:
#Fit the grid search on the training data
grid_search.fit(Xtrain, Ytrain)
#Gets the best parameters
best_params = grid_search.best_params_
best_score = np.sqrt(-grid_search.best_score_)

print("Best Parameters:", best_params)
print("Best CV RMSE: %.4f" % best_score)
#gets the best model
best_model = grid_search.best_estimator_
stack_y_pred = best_model.predict(Xtest)
stack_test_score = best_model.score(Xtest, Ytest)

print(f"Test: {stack_test_score}")
print(f"""
Mean Absolute Error = {mean_absolute_error(stack_y_pred, Ytest)},
Mean Squared Error = {mean_squared_error(stack_y_pred, Ytest)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(stack_y_pred, Ytest))},
R2 Score = {r2_score(stack_y_pred, Ytest)}
""")

print(f"Test Score: {stack_test_score}")

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:24:55] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "gb__learning_rate", "gb__max_depth", "gb__n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:24:56] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "gb__learning_rate", "gb__max_depth", "gb__n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:24:57] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "gb__learning_rate", "gb__max_depth", "gb__n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:24:58] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "gb__learning_rate", "gb__max_depth", "gb__n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
/usr

Best Parameters: {'gb__learning_rate': 0.01, 'gb__max_depth': 3, 'gb__n_estimators': 100}
Best CV RMSE: 1.4760
Test: 0.9561062548991788

Mean Absolute Error = 1.0751832135225259,
Mean Squared Error = 2.1755292620240234,
Root Mean Squared Error = 1.4749675460917855,
R2 Score = 0.954425974140603

Test Score: 0.9561062548991788


In [292]:
#Stacking regressor to combine all the regressors used
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
model4 = LinearRegression()
all_models = [('gb', model2), ('lr', model4)]
meta_model = LinearRegression()
stacking_model = StackingRegressor(estimators=all_models, final_estimator=meta_model, cv=5)
stacking_model.fit(Xtrain,Ytrain)
stack_test_score = stacking_model.score(Xtest, Ytest)
stack_y_pred = stacking_model.predict(Xtest)

print(f"""
Mean Absolute Error = {mean_absolute_error(stack_y_pred,Ytest)},
Mean Squared Error = {mean_squared_error(stack_y_pred,Ytest)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(stack_y_pred,Ytest))},
R2 Score = {r2_score(stack_y_pred, Ytest)}
          """)
print(f"Test Score: {stack_test_score}")



Mean Absolute Error = 1.320989451123486,
Mean Squared Error = 3.0668703146625775,
Root Mean Squared Error = 1.751248216176844,
R2 Score = 0.9341223336306049
          
Test Score: 0.9381224485466894


# Question 5

In [293]:
player_22 = pd.read_csv('/content/drive/MyDrive/FIFA 2023/players_22.csv')

<ipython-input-293-c28711c7bce8>:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  player_22 = pd.read_csv('/content/drive/MyDrive/FIFA 2023/players_22.csv')


In [294]:
testX = player_22[['movement_reactions','potential','passing','wage_eur',
             'value_eur','dribbling','attacking_short_passing','mentality_composure',
             'mentality_vision', 'international_reputation']]

In [295]:
testy =  player_22[['overall']]

In [297]:
test_numeric_data= testX.select_dtypes(include=['int64','float64'])

In [298]:
sc_test = SimpleImputer(strategy='median')
test_numeric_data_filled = sc_test.fit_transform(test_numeric_data)

In [299]:
test_numeric_data_df = pd.DataFrame(test_numeric_data_filled, columns=test_numeric_data.columns)

In [300]:
testX = test_numeric_data_df

In [301]:
testy = player_22[['overall']]

In [302]:
a = pd.concat([testy,testX], axis = 1)

In [303]:
Xtrain1,Xtest1,Ytrain1,Ytest1 = train_test_split(testX,testy,test_size=0.2, random_state=42)

In [307]:
best_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gb__learning_rate=0.01, gb__max_depth=3,
             gb__n_estimators=100, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, ...)

In [308]:
test_y_pred = model1.predict(Xtest1)
print(f"""
Mean Absolute Error = {mean_absolute_error(Ytest1, test_y_pred)},
Mean Squared Error = {mean_squared_error(Ytest1, test_y_pred)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(Ytest1, test_y_pred))},
R2 Score = {r2_score(Ytest1, test_y_pred)}
          """)


Mean Absolute Error = 1.0875320434570312,
Mean Squared Error = 2.196666435959827,
Root Mean Squared Error = 1.4821155271974675,
R2 Score = 0.9530608050654242
          


In [309]:
import joblib
joblib.dump(best_model, 'best_performing_model.pkl', compress = 9)

['best_performing_model.pkl']